In [2]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import itertools
from skimage import io
from sklearn import preprocessing
import glob

/home/niharika/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
data = pd.read_excel('Heirarchical_cluster/new_data/Daten_Ings/Kontext Phys-Ing_Kinematikgraphen_Rec 01.xlsx')

In [4]:
data.head()

,ParticipantName,MediaName,GazeEventDuration,AOI[Rectangle]Hit,AOI[Rectangle 2]Hit,AOI[Rectangle 3]Hit,AOI[Rectangle 4]Hit,AOI[Rectangle 5]Hit,AOI[Rectangle 6]Hit,AOI[Polygon]Hit,...,AOI[Rectangle 2]Hit.1,AOI[Rectangle 3]Hit.1,AOI[Rectangle 4]Hit.1,AOI[Rectangle 5]Hit.1,AOI[Rectangle 6]Hit.1,AOI[Polygon]Hit.1,AOI[Polygon 2]Hit.1,AOI[Polygon 3]Hit,AOI[Polygon 4]Hit.1,AOI[Polygon 5]Hit
0,BAJUKI03,Willkommen_Page_1.png,192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BAJUKI03,Willkommen_Page_1.png,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BAJUKI03,Willkommen_Page_1.png,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BAJUKI03,Willkommen_Page_1.png,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BAJUKI03,Willkommen_Page_1.png,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['ParticipantName', 'MediaName', 'GazeEventDuration',
       'AOI[Rectangle]Hit', 'AOI[Rectangle 2]Hit', 'AOI[Rectangle 3]Hit',
       'AOI[Rectangle 4]Hit', 'AOI[Rectangle 5]Hit', 'AOI[Rectangle 6]Hit',
       'AOI[Polygon]Hit', 'AOI[Polygon 2]Hit', 'AOI[Polygon 4]Hit',
       'AOI[Rectangle 7]Hit', 'AOI[Rectangle]Hit.1', 'AOI[Rectangle 2]Hit.1',
       'AOI[Rectangle 3]Hit.1', 'AOI[Rectangle 4]Hit.1',
       'AOI[Rectangle 5]Hit.1', 'AOI[Rectangle 6]Hit.1', 'AOI[Polygon]Hit.1',
       'AOI[Polygon 2]Hit.1', 'AOI[Polygon 3]Hit', 'AOI[Polygon 4]Hit.1',
       'AOI[Polygon 5]Hit'],
      dtype='object')

# FETCH ALL DATA FROM CSV FILES AND GET AOI SEQUENCES

In [6]:
class GetData():
    def __init__(self,filename):
        self.file = filename
        self.participant_names = []
        self.names = []
        self.seq = []
        self.frequency = {}
        self.gaze = {}
        self.final_sequence = []
    
    def get_names(self,file): #fetch names of all participants
        data = pd.read_excel(file)
        self.participant_names = data['ParticipantName'][0]
        return self.participant_names
    
    def get_files(self,filename): #fetch all csv files
        for f in filename:
            self.names.append(self.get_names(f))

    #read all csv files from the folder
    def get_file(self,file):
        page1 = ['PSQN_Page_1.png']
        page2 = ['PSQL_Page_1.png']
        data = pd.read_excel(file)

        data = data[['MediaName','GazeEventDuration','AOI[Rectangle]Hit', 'AOI[Rectangle 2]Hit', 'AOI[Rectangle 3]Hit',
           'AOI[Rectangle 4]Hit', 'AOI[Rectangle 5]Hit', 'AOI[Rectangle 6]Hit',
           'AOI[Polygon]Hit', 'AOI[Polygon 2]Hit', 'AOI[Polygon 4]Hit',
           'AOI[Rectangle 7]Hit', 'AOI[Rectangle]Hit.1', 'AOI[Rectangle 2]Hit.1',
           'AOI[Rectangle 3]Hit.1', 'AOI[Rectangle 4]Hit.1',
           'AOI[Rectangle 5]Hit.1', 'AOI[Rectangle 6]Hit.1', 'AOI[Polygon]Hit.1',
           'AOI[Polygon 2]Hit.1', 'AOI[Polygon 3]Hit', 'AOI[Polygon 4]Hit.1',
           'AOI[Polygon 5]Hit']]
        data_psqn = data[data.MediaName.isin(page1)]

        data_psql = data[data.MediaName.isin(page2)]
        data_psqn.columns = ['MediaName','GazeEventDuration','A', 'B','C','D','E','F','G','H','I',
                        'J','K','L','M',
                        'N','O','P','Q','R','S','T','U']
        data_psql.columns = ['MediaName','GazeEventDuration','A', 'B','C','D','E','F','G','H','I',
                        'J','K','L','M',
                        'N','O','P','Q','R','S','T','U']
        return data_psqn,data_psql
    
    #remove redundancies from scanpath
    def cleaned_path(self,x):
        length= len(x)
        i=0
        while(i<length):
            #print(i,x[i])
            area = x[i][0]
            duration = x[i][1]
            c = i+1
            total_duration = duration
            flag = False
            try:
                while(x[c][0]==area):
                    flag = True
                    total_duration = total_duration + x[c][1]
                    i=c+1
                    c= c+1
            except:
                continue

            self.final_sequence.append((area,total_duration))
            if flag == False:
                i=i+1
        return self.final_sequence
    
    #get the duration of each AOI in a scanpath sequence
    def get_gazedurtion(self,seq):
        for s in seq:
            #print(s)
            if s[0] in self.gaze.keys():
                self.gaze[str(s[0])]+=s[1]
            else:
                self.gaze[str(s[0])]=s[1]
        return self.gaze
    
    #get the number of times an AOI has occured in a scanpath sequence for each participant
    def get_frequence(self,seq):
        for s in seq:
            if s[0] in self.frequency.keys():
                self.frequency[str(s[0])]+=1
            else:
                self.frequency[str(s[0])]=1
        return self.frequency
    
    #function to generate scanpath sequence of each participant from the csv file
    def get_sequence(self,data):
        #seq = []
        GazeEventDuration = []
        for index, row in data.iterrows():
            if row['A'] == 1:
                self.seq.append(('A',row['GazeEventDuration']))
            elif row['B'] == 1:
                self.seq.append(('B',row['GazeEventDuration']))
            elif row['C'] == 1:
                self.seq.append(('C',row['GazeEventDuration']))
            elif row['D'] == 1:
                self.seq.append(('D',row['GazeEventDuration']))
            elif row['E'] == 1:
                self.seq.append(('E',row['GazeEventDuration']))
            elif row['F'] == 1:
                self.seq.append(('F',row['GazeEventDuration']))
            elif row['G'] == 1:
                self.seq.append(('G',row['GazeEventDuration']))
            elif row['H'] == 1:
                self.seq.append(('H',row['GazeEventDuration']))
            elif row['I'] == 1:
                self.seq.append(('I',row['GazeEventDuration']))
            elif row['J'] == 1:
                self.seq.append(('J',row['GazeEventDuration']))
            elif row['K'] == 1:
                self.seq.append(('K',row['GazeEventDuration']))
            elif row['L'] == 1:
                self.seq.append(('L',row['GazeEventDuration']))
            elif row['M'] == 1:
                self.seq.append(('M',row['GazeEventDuration']))
            elif row['N'] == 1:
                self.seq.append(('N',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['O'] == 1:
                self.seq.append(('O',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['P'] == 1:
                self.seq.append(('P',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['Q'] == 1:
                self.seq.append(('Q',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['R'] == 1:
                self.seq.append(('R',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['S'] == 1:
                self.seq.append(('S',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['T'] == 1:
                self.seq.append(('T',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
            elif row['U'] == 1:
                self.seq.append(('U',row['GazeEventDuration']))
                #GazeEventDuration.append(row['GazeEventDuration'])
  
        return self.seq

    
    
def main():
    paths1 = []
    paths2 = []
    frequence1 = []
    frequence2 = []
    t_gaze1 = []
    t_gaze2 = []
    total_gaze1 =[]
    total_gaze2 =[]
    names = []
    getdata = GetData(filename)
    for f in filename:
        #names.append(getdata.get_names(f))
        data_psqn,data_psql = getdata.get_file(f)
        #print(data1.head())
        #break
        seq1 = getdata.get_sequence(data_psqn)
        seq2 = getdata.get_sequence(data_psql)
        #print(seq1)
        #break

        freq1 = getdata.get_frequence(seq1)
        freq2 = getdata.get_frequence(seq2)
        #print(freq1)
        

        path1 = getdata.cleaned_path(seq1)
        path2 = getdata.cleaned_path(seq2)
        

        gaze1 = getdata.get_gazedurtion(seq1)
        gaze2 = getdata.get_gazedurtion(seq2)
        

        t_gaze1.append(gaze1)
        t_gaze2.append(gaze2)
        #print('gaze1 is ',gaze1)

        total_gaze1.append(sum(gaze1.values()))
        total_gaze2.append(sum(gaze2.values()))


        paths1.append(path1)
        paths2.append(path2)

        frequence1.append(freq1)
        frequence2.append(freq2)

#main()

In [8]:
# string1 = []
# string2 = []
# for i in range(len(paths1)):
#     #print(paths[i])
#     l= [x for (x, y) in paths1[i]]
    
#     s = ''.join(l)
    
#     string1.append(s)
    
# for i in range(len(paths2)):
#     #print(paths[i])
#     l= [x for (x, y) in paths2[i]]
    
#     s = ''.join(l)
    
#     string2.append(s)

# FUNCTION TO CREATE DOTPLOT FOR ALL PARTICIPANT PAIR

In [9]:
#create dotplot using the index and scanpath sequences

def create_dotplot(indexs):
    fcluster = []
    index_ = []
    psql0_dfs = {}
    all = {}
    memory = []
    participant_length=[]
    y_axis = 0
    x_axis = 0
    x_data = []
    for i in range(0,len(indexs),1):
        y_data = []
        y_data.append(y_axis)
        y_axis += len(paths1[indexs[i]])
        y_data.append(y_axis)
        x_axis = 0
        for j in range(0,len(indexs),1):
            x_data = []
            x_data.append(x_axis)
            x_axis += len(paths1[indexs[j]])
            x_data.append(x_axis)
            print()
            id_ = str(i)+'_'+str(j)
            id_reverse = str(j)+'_'+str(i)
            #print(id_)

            arr = np.zeros((len(paths1[indexs[j]]),len(paths1[indexs[i]])),dtype=int)
            print(arr.shape)
            df2 = pd.DataFrame(arr,columns=[a for (a,g) in paths1[indexs[i]]])
            df2.index = [a for (a,g) in paths1[indexs[j]]]
            all[id_] = df2.shape
            #print(all)
            if not i==j and id_ not in memory and id_reverse not in memory:
                print('creating dotplot')
                participant_length.append((y_data,x_data))
                memory.append(id_)
                memory.append(id_reverse)
                r=0
                #try:
                for index, row in df2.iterrows():
                    c = 0
                    #print(r,c)
                    while(c<len(paths1[indexs[i]])-1):
                        #print(paths2[psql0_index[i]][c][0],paths2[psql0_index[j]][r][0])
                        if paths1[indexs[i]][c][0] == paths1[indexs[j]][r][0]:
                            aoi = paths1[indexs[i]][c][0]
                            df2.iloc[[r], [c]] = (t_gaze1[indexs[j]][aoi] * t_gaze1[indexs[i]][aoi])/(total_gaze1[indexs[i]]*total_gaze1[indexs[j]])

                        c = c+1
                    r = r+1
            

            if i==j==0:
                psql0_dfs[i] = df2.T
            elif j==0 and i>0: #column should match
                if psql0_dfs[i-1].shape[1] == df2.shape[1]:
                    psql0_dfs[i] = df2
                else:
                    psql0_dfs[i] = df2.T

            else:# row match with base and column with previous
                if df2.shape[0] == psql0_dfs[i].shape[0]:
                    #print(psql1_dfs[i].shape,df2.shape)
                    psql0_dfs[i] = pd.concat([psql0_dfs[i], df2.T], axis=1)
                else:
                    psql0_dfs[i] = pd.concat([psql0_dfs[i], df2.T], axis=1)
              
            del df2



# SUB-MODULES

In [10]:
class Functions_():
    def __init__(self):
        self.f=[]
    
    def subdata(self,df_norm,region):
        X =[]
        y = []
        for i in range(len(df_norm.index)):
            for j in range(len(df_norm.columns)):
                #print(i,j)
                #cell_data = {}
                #weight = []
                #coordinates = []
                if df_norm.iloc[i,j] > 0 and i>region[0] and i<region[1] and j>region[2] and j<region[3]:
                    X.append(j)
                    y.append(i)
        return X,y


    def points_(self,coef,intercept,df_norm):
        points = []
        for i in range(len(df_norm.index)):
            for j in range(len(df_norm.columns)):
                #print(i,j)
                try:
                    if df_norm.iloc[i,j] > 0:
                        y = int(j*coef + intercept)
                        #print(y,i,(i,j))
                        if y == i or abs(y-i)<1:
                            #print('found ',y,i)
                            points.append((i,j))
                except:
                    print('out of bounds')

        return points

    def getlengths(self,cluster):
        #length = []
        length = [len(x) for x in cluster]
        length = list(set(length))
        return length  

    #perform regression on dotplots
    def regression(self,df_norm):
        cluster = []
        lines = []
        for l in participant_length:
            y0 = l[0][0]
            y1 = l[0][1]
            x0 = l[1][0]
            x1 = l[1][1]
            distance = 0.66*(y1-y0)
            #print(distance)
            for i in range(x0,x1,10):
                for j in range(y0,y1,10):
                    X,y= self.subdata(df_norm,[j,j+distance,i,i+distance])
                    X = np.array(X)
                    y = np.array(y)
                    #print(X.shape,X,y.shape,y)
                    try:
                        reg = LinearRegression().fit(X.reshape(-1,1), y)
                        points = self.points_(reg.coef_,reg.intercept_,df_norm)
                        lines.append((reg.coef_,reg.intercept_))
                        cluster.append(points)
                        print(points)
                    except:
                        print('dskncskdj')
        return cluster,lines

    def longestSubstringFinder(self,string1,weight1,a, string2,weight2,b):
        answer = ""
        len1, len2 = len(string1), len(string2)
        coordinates = []
        for i in range(len1):
            match = ""
            val = 0

            for j in range(len2):
                if (i + j < len1 and string1[i + j] == string2[j]):
                    match += string2[j]
                    val += weight1[i]+weight2[j]
                    coordinates.append(a[i])
                    coordinates.append(b[j])
                    #print(val)
                else:
                    if (len(match) > len(answer)): answer = match
                    match = ""
                    #val = 0
        return answer,val,list(set(coordinates))

    def get_string(self,stri):
        string = []
        weight = []
        for (m,n) in stri:
            string.append(base.columns[n]) #use df_norm name
            weight.append(base.iloc[m][n])
        return ''.join(string),weight


    def compute_dist(self,a,b):
        match,weight = get_string(a)
        match_,weight_ = get_string(b)
        matchstring,val,coordinates = longestSubstringFinder(match,weight,a,match_,weight_,b)
        return [len(matchstring),val,matchstring,coordinates]


    #compute distances
    def compute_dist_dict(self,fcluster__):
        distance = {}
        for i in range(0,len(fcluster__),1):
            each_dist = {}
            for j in range(0,len(fcluster__),1):
                #print(i,j)
                if i==j:
                    dist = [0,0,'']
                    #continue
                else:
                    dist = compute_dist(fcluster__[i],fcluster__[j])
                each_dist[j] = dist
            #print(each_dist)
            distance[i] = each_dist
        return distance

    #remove redundancy
    def remove_redundancy(self,fcluster_):
        fcluster__=[]
        for f in fcluster_:
            x = []
            #get unique x coordinates
            x_coord = [x for (x,y) in f]
            #print(set(x_coord))
            for xc in x_coord:
                #print(xc)
                unique_x = [i for i in f if i[0]==xc]
                #print(unique_x)
                if len(unique_x)>1:
                    for j in unique_x:
                        #all_x = [item for item in f if item[0] == unique_x]
                        val = [df_norm.iloc[v[0]][v[1]] for v in unique_x]
                        max_index = val.index(max(val))
                        x.append(unique_x[max_index])
                        #print(val,))
                        #print(j,df_norm.iloc[j[0]][j[1]])

                        #print([df_norm.iloc[v[0]][v[1]] for v in all_x])
                else:
                    x.append(unique_x[0])
            fcluster__.append(list(set(x)))  
        return fcluster__
    


    def H_cluster(self,fcluster__): #heirarchical clustering
        common_seq = []
        while(len(fcluster__) >0):
            print(len(fcluster__))
            distance = self.compute_dist_dict(fcluster__) 
            #print(distance)
            bestmatch = {}
            for i in range(len(distance.keys())):
                new_data = {k: v for k, v in distance[i].items() if not v[1] == 0}
                #print('new_data',new_data)
                if len(new_data)>0:
                    bestmatch[i] = max(new_data.items(), key = lambda k : k[1])
            print('best match is ',max(bestmatch.items(), key = lambda k : k[1]))
            mergeitems = max(bestmatch.items(), key = lambda k : k[1])
            common_seq.append([mergeitems[0],mergeitems[1][0],mergeitems[1][1][2]])
            print(common_seq)
            #fcluster_.pop(mergeitems[0])
            #fcluster_.pop(mergeitems[1][0])
            remove_indices = [mergeitems[0],mergeitems[1][0]]
            fcluster__ = [i for j, i in enumerate(fcluster__) if j not in remove_indices]
            fcluster__.append(mergeitems[1][1][3])
            #print(fcluster_)
        return common_seq

# main function

In [21]:
paths1 = []
paths2 = []
frequence1 = []
frequence2 = []
t_gaze1 = []
t_gaze2 = []
total_gaze1 =[]
total_gaze2 =[]
names = []
psql0_index = [] # particiapnts who scored 0 in psql
psqn0_index = [] # particiapnts who scored 0 in psqn
psql1_index = [] # particiapnts who scored 1 in psql
psqn1_index = [] # particiapnts who scored 1 in psqn
num = 3  
threshold = 0
len_threshold = 6

#fetch names of all participants
def get_names(file):
    data = pd.read_excel(file)
    name = data['ParticipantName'][0]
    return name


    
def get_scores(names):
    #get index of all participants on the basis of scores and store them in above list
    for n in names:
        index = grade.loc[grade['Participant'] == n, 'PSQL'].index.item()
        psql = grade.loc[grade['Participant'] == n, 'PSQL'].iloc[0]
        psqn = grade.loc[grade['Participant'] == n, 'PSQN'].iloc[0]
        if psql == 0:
            psql0_index.append(index)
        else:
            psql1_index.append(index)
        if psqn == 0:
            psqn0_index.append(index)
        else:
            psqn1_index.append(index)
    
def main_(num,threshold,len_threshold):
    filename = glob.glob("/home/niharika/notebooks/Heirarchical_cluster/new_data/Daten_Ings/*")
    for f in filename:
        names.append(get_names(f))
    print(names)
    grade = pd.read_excel('Heirarchical_cluster/new_data/Korrektheit_PSQL_PSQN.xlsx',sheet_name='Tabelle2')
    grade.index=grade['Participant']
    grade = grade.reindex(names)
    grade.reset_index(drop=True, inplace=True)
    

    getdata = GetData(filename)
    for f in filename:
        print(f)
        #names.append(getdata.get_names(f))
        data_psqn,data_psql = getdata.get_file(f)
        #print(data1.head())
        #break
        seq1 = getdata.get_sequence(data_psqn)
        seq2 = getdata.get_sequence(data_psql)
        #print(seq1)
        #break

        freq1 = getdata.get_frequence(seq1)
        freq2 = getdata.get_frequence(seq2)
        #print(freq1)
        

        path1 = getdata.cleaned_path(seq1)
        path2 = getdata.cleaned_path(seq2)
        

        gaze1 = getdata.get_gazedurtion(seq1)
        gaze2 = getdata.get_gazedurtion(seq2)
        

        t_gaze1.append(gaze1)
        t_gaze2.append(gaze2)
        #print('gaze1 is ',gaze1)

        total_gaze1.append(sum(gaze1.values()))
        total_gaze2.append(sum(gaze2.values()))


        paths1.append(path1)
        paths2.append(path2)

        frequence1.append(freq1)
        frequence2.append(freq2)
        
    


    get_scores(names)
    indexs = list(set(psql0_index))
    indexs = indexs[:num]
    print('indexs ',indexs)
    create_dotplot(indexs)
    
    #concatenate all dotplots
    for i in range(0,len(indexs),1):
        if i==0:
            base = psql0_dfs[i]
        else:
            base = pd.concat([base,psql0_dfs[i]], axis=0)
            
    x = base.values #returns a numpy array
    #min_max_scaler = preprocessing.MinMaxScaler()
    #x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x)
    df_norm = df.apply(lambda x: np.where(x==threshold,0,x)) #replace values in dotplot
    cluster,lines = regression(df_norm)
    clusterf =[]
    for c in cluster:
        #print(c)
        if not len(c)==0:
            if not len(set([c1[0] for c1 in c])) == 1:
                #print('found ',c)
                clusterf.append(c)
    fcluster = []
    cluster_refined = list(k for k,_ in itertools.groupby(clusterf))
    c = []
    for line in cluster_refined:
        if len(line)>5:
            c.append(line)
    fcluster.append(c)
    modules_ = Functions()
    fcluster_ = fcluster[0]
    fcluster__ = modules_.remove_redundancy(fcluster_)
    fcluster__ = [v for v in fcluster__ if len(v)>len_threshold]
    common_seq = modules_.H_cluster(fcluster__)
    print(common_seq)

In [1]:
main_(3,0,6)